In [0]:
from pyspark.sql import SparkSession, functions as F, types as T, Window
from delta.tables import DeltaTable
import os

In [0]:
from pyspark.sql import types as T

catalog_table = "production.refined.d_hospedes"

if not spark.catalog.tableExists(catalog_table):
    schema = T.StructType([
        T.StructField("pk_hospede", T.StringType(), True),
        T.StructField("sk_hospede", T.LongType(), True),
        T.StructField("nome", T.StringType(), True),
        T.StructField("data_nascimento", T.DateType(), True),
        T.StructField("sexo", T.StringType(), True),
        T.StructField("nacionalidade", T.StringType(), True),
        T.StructField("tipo_cliente", T.StringType(), True),
        T.StructField("email", T.StringType(), True),
        T.StructField("telefone", T.StringType(), True),
        T.StructField("idade", T.IntegerType(), True),        
        T.StructField("start_date", T.DateType(), True),
        T.StructField("update_date", T.DateType(), True)
    ])

    df_empty = spark.createDataFrame([], schema)

    # Cria tabela Delta no catálogo
    df_empty.write.format("delta").saveAsTable(catalog_table)

    print("Tabela Delta criada com sucesso em:", catalog_table)
else:
    print("Tabela já existe:", catalog_table)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from delta.tables import DeltaTable

# --- Parâmetros ---
catalog_table = "production.refined.d_hospedes"
source_table = "production.trusted.tb_hospedes"

print("Iniciando processo de merge para a tabela:", catalog_table)

# --- 1. Carregar Tabela de Destino ---
try:
    delta_table = DeltaTable.forName(spark, catalog_table)
    df_dim_existente = delta_table.toDF()
    is_initial_load = df_dim_existente.count() == 0
except Exception:
    is_initial_load = True
    print(f"Tabela {catalog_table} não encontrada. Assumindo carga inicial.")

# --- 2. Carregar e Preparar Dados de Origem ---
# Lê todas as colunas necessárias da origem e calcula a chave primária (pk).
df_source = (
    spark.read.table(source_table)
    .select(
        "hospede_id", "nome", "data_nascimento", "sexo", "nacionalidade",
        "tipo_cliente", "email", "telefone", "idade"
    )
    .dropDuplicates(["hospede_id"])
    .withColumn(
        "pk_hospede",
        F.sha2(F.concat_ws("||", F.col("hospede_id")), 256)
    )
)

# --- 3. Identificar e Preparar Apenas os Novos Registros ---
if is_initial_load:
    df_novos_hospedes = df_source
else:
    # Isola apenas os registros que são realmente novos
    df_novos_hospedes = df_source.join(
        df_dim_existente.select("pk_hospede"),
        on="pk_hospede",
        how="left_anti"
    )

# Calcula o último sk_hospede para continuar a sequência.
if is_initial_load:
    last_id = 0
else:
    last_id_row = df_dim_existente.agg(F.max("sk_hospede")).collect()
    last_id = last_id_row[0][0] if last_id_row and last_id_row[0][0] is not None else 0

# Gera novas SKs e metadados somente para os novos registros.
window = Window.orderBy("pk_hospede")
df_com_novas_sks = (
    df_novos_hospedes.withColumn("sk_hospede", (F.row_number().over(window) + last_id).cast(T.LongType()))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast(T.DateType()))
)

# --- 4. Criar o DataFrame Final para o MERGE ---
# Unimos o df_source (com todos os registros) com as novas SKs (apenas para novos registros).
# O resultado é um DataFrame com todos os registros de origem, onde os novos têm sk_hospede
# e os antigos têm sk_hospede nulo (o que não importa, pois eles irão para o UPDATE).
df_final_source = df_source.join(
    df_com_novas_sks.select("pk_hospede", "sk_hospede", "start_date", "update_date"),
    "pk_hospede",
    "left"
).sort("sk_hospede")

# --- 5. Executar a Operação de MERGE ---
# Condição para atualizar: apenas se dados descritivos mudarem.
update_condition = "target.nome <> source.nome OR target.email <> source.email OR target.nacionalidade <> source.nacionalidade OR target.telefone <> source.telefone OR target.tipo_cliente <> source.tipo_cliente OR target.idade <> source.idade"
(
    delta_table.alias("target")
    .merge(
        df_final_source.alias("source"),
        "target.pk_hospede = source.pk_hospede"
    )
    .whenMatchedUpdate(
        condition=update_condition,
        set={
            "nome": F.col("source.nome"), 
            "data_nascimento": F.col("source.data_nascimento"),
            "sexo": F.col("source.sexo"), 
            "nacionalidade": F.col("source.nacionalidade"),
            "tipo_cliente": F.col("source.tipo_cliente"), 
            "email": F.col("source.email"),
            "telefone": F.col("source.telefone"), 
            "idade": F.col("source.idade"),
            "update_date": F.current_date()
        }
    )
    .whenNotMatchedInsert(
        values={
            "pk_hospede": F.col("source.pk_hospede"),
            "sk_hospede": F.col("source.sk_hospede"), # Agora esta coluna existe no df_final_source
            "nome": F.col("source.nome"),
            "data_nascimento": F.col("source.data_nascimento"),
            "sexo": F.col("source.sexo"),
            "nacionalidade": F.col("source.nacionalidade"),
            "tipo_cliente": F.col("source.tipo_cliente"),
            "email": F.col("source.email"),
            "telefone": F.col("source.telefone"),
            "idade": F.col("source.idade"),
            "start_date": F.col("source.start_date"),
            "update_date": F.col("source.update_date")
        }
    )
).execute()

print(f"Merge/upsert concluído com sucesso na tabela: {catalog_table}")

In [0]:
%sql
select DISTINCT sexo from production.refined.d_hospedes

